In [380]:
import os
import requests
import re
import pandas as pd

from html.parser import HTMLParser
from chatgpt_poc.csv import read_csv, write_csv, write_text, read_text
from chatgpt_poc.text_completion import request_completion
from chatgpt_poc import cparser

In [327]:
# data
home = '/home/nroshania/git/ChatGPT-POC/'
cparser.read(f'{home}conf/pipeline.conf')

header, *data = read_csv(f'{home}tests/data/wdod/requests/wdod_request_urls_040322.csv')
r = [(d[1], d[2]) for d in data] # requests

In [328]:
r[0:2]

[('80a2d56a-757c-4eef-8eaf-1e4840c2ac55',
  'https://www.google.com/search?num=5&q=https%3A%2F%2Fwww.linkedin.com%2Fin%2Fvivek-dubey-4a05b0238+%22-+Present%22'),
 ('4160375',
  'https://www.google.com/search?num=50&q=Adam+Home+Zenith+Wealth+Creation+site%3Alinkedin.com')]

In [336]:
# make requests and store html in seperate files
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

for record_id, url in r:
    t = requests.get(url, headers=headers)
    write_text(file_loc=f'{home}tests/data/wdod/responses/{record_id}.txt', data=t.text)
#     break

### HTML parser

In [396]:
resp = read_text(file_loc=f'{home}tests/data/wdod/responses/4160375.txt')
resp = re.sub('<[\/em]+>', '', resp)

In [397]:
class WDODParser(HTMLParser):
    
    def __init__(
            self,
    ):
        super().__init__()
        self.found_links = set()
        self.titles = set()
        self.subtitles = set()
        # self.hierarchy = []
        self.foundh3 = False
        self.foundspan = False

    def handle_starttag(self, tag, attrs):
        
        # mark headers - this can be abstracted
        if tag in ('h3'):
            self.foundh3 = True
        
        if tag in ('span'):
            self.foundspan = True
            
        # gather links within tags
        for attr, url in attrs:
            # self.hierarchy.append((self.getpos(), attr, url))
            if attr != "href":
                continue
            else:
#                 print(url)
                if isinstance(re.search('(https:\/\/[\w\W]+.linkedin.com[\w\W]+)', url), re.Match) and ('google.com' not in url):
                    self.found_links.add((self.getpos(), re.search('(https:\/\/[\w\W]+.linkedin.com[\w\W]+)', url).group(1)))

    def handle_endtag(self, tag):
        if tag in ('h3'):
            self.foundh3 = False
        if tag in ('span'):
            self.foundspan = False

    def handle_data(self, data):
#         print(data)
        if self.foundh3:
            self.titles.add((self.getpos(), data)) # titles
        if self.foundspan:
            # locations
            self.subtitles.add((self.getpos(), data))
        

parser = WDODParser()
parser.feed(resp)

In [ ]:
# data extraction logic

# order data objects by line number
titles = sorted([(t[0][0], t[0][1], t[1])for t in parser.titles], key=lambda line: line[1])
urls = sorted([(t[0][0], t[0][1], t[1]) for t in parser.found_links], key=lambda line: line[1])

res = list(zip(titles, urls))
results = list(zip([t[2] for t in titles], [u[2] for u in urls]))

# write_csv(file_loc=f'{home}tests/data/wdod/parsed/{record_id}.csv', data=results)
# results[0:10]
res[0:15]

In [69]:
request_completion('yo what\'s the vibe?')

{'creation_id': 'cmpl-71KYAe9rhVOsb0ROlnuYppqR7vGKW',
 'model': 'text-davinci-003',
 'prompt_tokens': 6,
 'choices-text': ['\n\nThe vibe is one of energy and positivity!'],
 'success': True}

### BeautifulSoup

In [ ]:

import requests
from bs4 import BeautifulSoup
import re

# Define the search URL
url = "https://www.google.com/search?num=50&q=Adam+Home+Zenith+Wealth+Creation+site%3Alinkedin.com"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
search_results = soup.find_all( class_='g')

for result in search_results:
    title = result.find('h3').get_text()
    print('title: ', title)
    print()

    url = result.find('a')['href']
    print('url: ', url)
    print()

    blurb= result.find(style = '-webkit-line-clamp:2').get_text()
    print('blurb:', blurb)
    print()
    
    subtitles= result.find_all('span')[9:17]
    subtitle_list = []
    for subtitle in subtitles:
        subtitle_list.append(subtitle.text.strip())
    if blurb == subtitle_list[0]:
        subtitle = None
    else:
        subtitle = subtitle_list

    print('subtitle: ', subtitle)
    print('------------------------------------------------------------------------------------------------------------------')